In [1]:
from scipy.signal import butter, lfilter
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
import datetime
import math
from zipfile import ZipFile

In [2]:
def get_utc_date_time(ts):
    return pd.to_datetime(ts, unit='s', infer_datetime_format = True, utc = True)
#.strftime('%H:%M:%S:%f')

def add_fs(sample_rate, date):
    return date + datetime.timedelta(milliseconds=1.0/(sample_rate) * 1000.0)

In [3]:
def time_range(df_length, start_timestamp, sample_rate):
    '''Generate date time range based on sample rate and starting timestamp'''
    time_range = []
    t_0 = get_utc_date_time(float(start_timestamp))
    time_range.append(t_0)
    
    next_date = t_0
    
    for count in range(df_length-1):
        next_date = add_fs(sample_rate, next_date)
        time_range.append(next_date)
    return time_range

In [4]:
def format_time(t):
    s = t.strftime('%Y-%m-%d %H:%M:%S.%f')
    return s[:-3]

In [5]:
# Fourier transform that takes a df with X,Y,Z columns and returns one with single column of value: sqrt(x^2 + y^2 + z^2)

In [6]:
def fourier_transform(df):
    '''fourier transformation: srqt(x^2 + y^2 + z^2)'''
    transformed_df = pd.DataFrame()
    transformed_df['Transformed'] = df.apply(lambda row: f_t(row), axis=1)
    print(transformed_df)
    return transformed_df

def f_t(row):
    return math.sqrt(row['X']**2 + row['Y']**2 + row['Z']**2)

In [7]:
base_path = r"C:\Users\shaye\Documents\EAT Lab\Sensor_Data\Empatica data"
participant_id = "PR025"
base_path

'C:\\Users\\shaye\\Documents\\EAT Lab\\Sensor_Data\\Empatica data'

In [8]:
list_of_zips = glob.glob(base_path + '\A*.zip')

In [ ]:
acc_all = []

for zip_file in list_of_zips:
    zf = ZipFile(zip_file)
    acc_df_raw = pd.read_csv(zf.open('ACC.csv'), dtype='int16') #nrows=1000
    
    starting_timestamp = acc_df_raw.columns[0]
    sample_rate = acc_df_raw[starting_timestamp][0]
    
    acc_df_raw = acc_df_raw.drop([0, 0]) #First row is the sample rate, we don't need it for this script
    
    #ft_df = fourier_transform(acc_df_raw)
    #new_columns = { ft_df.columns[0]: 'X', ft_df.columns[1]: 'Y', ft_df.columns[2]: 'Z' }
    #ft_df = ft_df.rename(columns=new_columns)
    #ft_df['ParticipantId'] = participant_id
    #time_col = time_range(len(ft_df),starting_timestamp,sample_rate)
    #ft_df['Timestamp'] = time_col
    #print(ft_df)
    #acc_all.append(ft_df)
    
    new_columns = { acc_df_raw.columns[0]: 'X', acc_df_raw.columns[1]: 'Y', acc_df_raw.columns[2]: 'Z' }
    acc_df_raw = acc_df_raw.rename(columns=new_columns)
    acc_df_raw['ParticipantId'] = participant_id
    
    time_col = time_range(len(acc_df_raw),starting_timestamp,sample_rate)
    acc_df_raw['Timestamp'] = time_col
    acc_df_raw['Timestamp'] = acc_df_raw.apply(lambda x: format_time(x['Timestamp']) + '+00:00', axis=1)
    
    acc_all.append(acc_df_raw)

In [ ]:
acc_df_all = []
acc_df_all = pd.concat(acc_all)

In [ ]:
acc_df_all